In [1]:
#import libraries
import pandas as pd
import numpy as np


In [2]:

# read in the data
data = pd.read_csv('adult.data', header=None)


In [3]:

# assign column names
data.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']


In [ ]:
# remove rows with missing values
data = data[(data.astype(str) != ' ?').all(axis=1)]


# Change the salary column to binary data instead of boolean conditions
data['salary_bi'] = data.apply(lambda row: 1 if '>50K'in row['salary'] else 0, axis=1)
data = data.drop(['salary', 'education', 'native-country', 'fnlwgt'], axis=1)


#data_num = data.drop(['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'], axis=1)

In [ ]:

# convert discrete variables to dummy variables
data = pd.get_dummies(data, columns=['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex'])

In [ ]:

# normalize numerical variables
data['age'] = (data['age'] - data['age'].min()) / (data['age'].max() - data['age'].min())
data['education-num'] = (data['education-num'] - data['education-num'].min()) / (data['education-num'].max() - data['education-num'].min())
data['capital-gain'] = (data['capital-gain'] - data['capital-gain'].min()) / (data['capital-gain'].max() - data['capital-gain'].min())
data['capital-loss'] = (data['capital-loss'] - data['capital-loss'].min()) / (data['capital-loss'].max() - data['capital-loss'].min())
data['hours-per-week'] = (data['hours-per-week'] - data['hours-per-week'].min()) / (data['hours-per-week'].max() - data['hours-per-week'].min())


In [ ]:
corr_matrix = data.corr()

corr_matrix['salary_bi'].sort_values(ascending=False)

In [ ]:

# split data into features and target
X = data.drop(columns='salary_bi')
y = data['salary_bi']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, bootstrap=True, oob_score=True, n_jobs=-1, random_state=42)
rnd_clf.fit(X,y)

In [ ]:
rnd_clf.oob_score_

In [ ]:
rnd_clf.feature_importances_